<a href="https://colab.research.google.com/github/teshi24/aiso/blob/main/Travelling_Salesperson_with_Precedences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travelling Salesperson with Precedence Constraints

Lucerne University of Applied Sciences and Arts - School of Information Technology

The following exercise originates from a recent research project.

Our industry partner is a Swiss small to medium sized company in the electricity market. They have one central headquarter from where they send out teams for various repair and maintenance tasks. Every task is bound to a specific location. The company wants to minimize the overall travelling costs. For simplicity we assume that the costs between any pair of locations are known in advance. However, they additionally have to respect certain inter-dependencies among the tasks. For example: for safety reasons it is important to first switch off electricity (task A) before starting the repair work on electric components (task B). This induces a constraint that task A must always be completed before task B can be started.

Write a constraint optimization model that takes the number of teams as input together with a costs matrix between any pair of locations. For testing we provide code that automatically creates cost matrices with random numbers. The following figure displays the inter-dependencies among 11 tasks to be worked off. We further assume that location 0 (not displayed here) refers to the company’s headquarter. The duration for each tasks can be considered a constant and can thus be neglected in your model.

Task inter-dependencies are shown in the following figure:

![image.png](attachment:image.png)

@author: Marc Pouly
@author: Tobias Mérinat

In [3]:
!pip install ortools tsp_examples

ERROR: Could not find a version that satisfies the requirement tsp_examples (from versions: none)
ERROR: No matching distribution found for tsp_examples


In [8]:
# @title

import sys
sys.path.append('../lecture-cpsat/')

from ortools.constraint_solver import pywrapcp, routing_enums_pb2
# Example input for Travelling Salesman
#
# @author: Tobias Mérinat and Marc Pouly

import copy
import numpy as np
from sklearn.metrics import DistanceMetric
from ortools.constraint_solver import pywrapcp


#  Some classes that represent different exercise types
class TspExample:
    """ abstract base class """

    def __init__(self, names, nodes, vehicles):
        self.names = names
        self.nodes = nodes
        self.vehicles = vehicles
        self.depot = 0
        self._manager = None

    def distance(self, a, b):
        raise NotImplementedError()

    def size(self):
        return len(self.names)

    def manager(self):
        if self._manager is None:
            self._manager = pywrapcp.RoutingIndexManager(self.size(), self.vehicles, self.depot)
        return self._manager


class Precomputed(TspExample):
    def distance(self, a, b):
        return self.nodes[self.manager().IndexToNode(a)][self.manager().IndexToNode(b)]


class Euclidean(TspExample):
    def distance(self, a, b):
        x = np.array((*self.nodes[self.manager().IndexToNode(a)], *self.nodes[self.manager().IndexToNode(b)])).reshape(2, 2)
        return DistanceMetric.get_metric('euclidean').pairwise(x)[0][1]


class Manhattan(TspExample):
    def distance(self, a, b):
        x = np.array((*self.nodes[self.manager().IndexToNode(a)], *self.nodes[self.manager().IndexToNode(b)])).reshape(2, 2)
        return DistanceMetric.get_metric('manhattan').pairwise(x)[0][1]


class GPS(TspExample):
    def distance(self, a, b):
        from math import radians
        # pairwise wants shape (n_samples, 2) lat lon in radians
        x = np.fromiter(map(radians, (*self.nodes[self.manager().IndexToNode(a)], *self.nodes[self.manager().IndexToNode(b)])), np.float).reshape(2, 2)
        dists = DistanceMetric.get_metric('haversine').pairwise(x)
        return 3959 * dists[0][1]  # multiply distance by earth radius in miles


# Various examples
def small(vehicles):
    return Precomputed(
        names=["New York", "Los Angeles", "Chicago", "Salt Lake City"],
        nodes=[
                [0, 2451, 713, 1018],   # New York
                [2451, 0, 1745, 1524],  # Los Angeles
                [713, 1745, 0, 355],    # Chicago
                [1018, 1524, 355, 0]],  # Salt Lake City
        vehicles=vehicles)


def large(vehicles):
    instance = Precomputed(
        names=["New York", "Los Angeles", "Chicago", "Minneapolis", "Denver", "Dallas", "Seattle", "Boston",
               "San Francisco", "St. Louis", "Houston", "Phoenix", "Salt Lake City"],
        nodes=[
                [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],  # New York
                [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],  # Los Angeles
                [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],  # Chicago
                [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],  # Minneapolis
                [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],  # Denver
                [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],  # Dallas
                [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],  # Seattle
                [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],  # Boston
                [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],  # San Francisco
                [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],  # St. Louis
                [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],  # Houston
                [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],  # Phoenix
                [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0]],  # Salt Lake City
        vehicles=vehicles)
    instance.capacities = [15] * instance.vehicles
    instance.demands = [5, 6, 4, 2, 2, 1, 2, 1, 2, 4, 4, 1, 3]
    instance.demand = lambda node: int(instance.demands[instance.manager().IndexToNode(node)])
    return instance


def gps(vehicles):
    return GPS(
        names=["New York", "Los Angeles", "Chicago", "Minneapolis", "Denver", "Dallas", "Seattle", "Boston",
               "San Francisco", "St. Louis", "Houston", "Phoenix", "Salt Lake City"],
        nodes=[
              (40.71, -74.01),  # New York
              (34.05, -118.24),  # Los Angeles
              (41.88, -87.63),  # Chicago
              (44.98, -93.27),  # Minneapolis
              (39.74, -104.99),  # Denver
              (32.78, -96.89),  # Dallas
              (47.61, -122.33),  # Seattle
              (42.36, -71.06),  # Boston
              (37.77, -122.42),  # San Francisco
              (38.63, -90.20),  # St.Louis
              (29.76, -95.37),  # Houston
              (33.45, -112.07),  # Phoenix
              (40.76, -111.89)],  # Salt Lake City
        vehicles=vehicles)


# Random values with parameters defined by the Routing exercise
def randomly(vehicles=2, n=12, seed=False):
    if seed:
        np.random.seed(42)
    random_costs = np.random.randint(1, 1000, (n, n))
    nodes = np.triu(random_costs, 1) + np.triu(random_costs, 1).T  # zero out lower half, then mirror upper
    return Precomputed(names=range(len(random_costs)), nodes=nodes, vehicles=vehicles)


def drilling(vehicles):
    return Euclidean(
        names=range(280),  # the length of the node list
        nodes=[(288, 149), (288, 129), (270, 133), (256, 141), (256, 157), (246, 157), (236, 169),
               (228, 169), (228, 161), (220, 169), (212, 169), (204, 169), (196, 169), (188, 169),
               (196, 161), (188, 145), (172, 145), (164, 145), (156, 145), (148, 145), (140, 145),
               (148, 169), (164, 169), (172, 169), (156, 169), (140, 169), (132, 169), (124, 169),
               (116, 161), (104, 153), (104, 161), (104, 169), (90, 165), (80, 157), (64, 157),
               (64, 165), (56, 169), (56, 161), (56, 153), (56, 145), (56, 137), (56, 129), (56, 121),
               (40, 121), (40, 129), (40, 137), (40, 145), (40, 153), (40, 161), (40, 169), (32, 169),
               (32, 161), (32, 153), (32, 145), (32, 137), (32, 129), (32, 121), (32, 113), (40, 113),
               (56, 113), (56, 105), (48, 99), (40, 99), (32, 97), (32, 89), (24, 89), (16, 97),
               (16, 109), (8, 109), (8, 97), (8, 89), (8, 81), (8, 73), (8, 65), (8, 57), (16, 57),
               (40, 83), (40, 73), (40, 63), (40, 51), (44, 43), (44, 35), (44, 27), (32, 25), (24, 25),
               (16, 25), (16, 17), (24, 17), (32, 17), (44, 11), (56, 9), (56, 17), (56, 25), (56, 33),
               (56, 41), (64, 41), (72, 41), (72, 49), (56, 49), (48, 51), (56, 57), (56, 65), (48, 63),
               (48, 73), (56, 73), (56, 81), (48, 83), (56, 89), (56, 97), (104, 97), (104, 105),
               (104, 113), (104, 121), (104, 129), (104, 137), (104, 145), (116, 145), (124, 145),
               (132, 145), (132, 137), (140, 137), (148, 137), (156, 137), (164, 137), (172, 125),
               (172, 117), (172, 109), (172, 101), (172, 93), (172, 85), (180, 85), (180, 77), (180, 69),
               (180, 61), (180, 53), (172, 53), (172, 61), (172, 69), (172, 77), (164, 81), (148, 85),
               (124, 85), (124, 93), (124, 109), (124, 125), (124, 117), (124, 101), (104, 89),
               (104, 81), (104, 73), (104, 65), (104, 49), (104, 41), (104, 33), (104, 25), (104, 17),
               (92, 9), (80, 9), (72, 9), (64, 21), (72, 25), (80, 25), (80, 25), (80, 41), (88, 49),
               (104, 57), (124, 69), (124, 77), (132, 81), (140, 65), (132, 61), (124, 61), (124, 53),
               (124, 45), (124, 37), (124, 29), (132, 21), (124, 21), (120, 9), (128, 9), (136, 9),
               (148, 9), (162, 9), (156, 25), (172, 21), (180, 21), (180, 29), (172, 29), (172, 37),
               (172, 45), (180, 45), (180, 37), (188, 41), (196, 49), (204, 57), (212, 65), (220, 73),
               (228, 69), (228, 77), (236, 77), (236, 69), (236, 61), (228, 61), (228, 53), (236, 53),
               (236, 45), (228, 45), (228, 37), (236, 37), (236, 29), (228, 29), (228, 21), (236, 21),
               (252, 21), (260, 29), (260, 37), (260, 45), (260, 53), (260, 61), (260, 69), (260, 77),
               (276, 77), (276, 69), (276, 61), (276, 53), (284, 53), (284, 61), (284, 69), (284, 77),
               (284, 85), (284, 93), (284, 101), (288, 109), (280, 109), (276, 101), (276, 93),
               (276, 85), (268, 97), (260, 109), (252, 101), (260, 93), (260, 85), (236, 85), (228, 85),
               (228, 93), (236, 93), (236, 101), (228, 101), (228, 109), (228, 117), (228, 125),
               (220, 125), (212, 117), (204, 109), (196, 101), (188, 93), (180, 93), (180, 101),
               (180, 109), (180, 117), (180, 125), (196, 145), (204, 145), (212, 145), (220, 145),
               (228, 145), (236, 145), (246, 141), (252, 125), (260, 129), (280, 133)],
        vehicles=vehicles)


def manhattan(vehicles=4):
    instance = Manhattan(
        names=range(17),  # the length of the node list
        nodes=[(4, 4), (2, 0), (8, 0), (0, 1), (1, 1), (5, 2), (7, 2), (3, 3), (6, 3),
               (5, 5), (8, 5), (1, 6), (2, 6), (3, 7), (6, 7), (0, 8), (7, 8)],
        vehicles=vehicles)
    instance.capacities = [15] * instance.vehicles
    instance.demands = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    instance.demand = lambda node: int(instance.demands[instance.manager().IndexToNode(node)])
    return instance

# The library tsp_example canot be installed with pip.
# It is contained in the folder with or-tools examples from my lecture.

The randomly example has the correct dimensions for this exercise. If you want values that change with every call,  omit the seed argument.

In [9]:
example = randomly(vehicles=2, n=12, seed=42)
manager = example.manager()
distance = example.distance
num_locations = example.size()
location_names = example.names
num_teams = example.vehicles

Create routing model

In [10]:
routing = pywrapcp.RoutingModel(manager)

All teams use the same cost model

In [11]:
routing.SetArcCostEvaluatorOfAllVehicles(routing.RegisterTransitCallback(distance))

Force all teams to process at least one task

In [13]:
# Write your code here ...
# Create dimension variables
routing.AddConstantDimension(1, num_locations, True, "count")
# Get dimension variables from model (access by name)
count = routing.GetDimensionOrDie("count")

for i in range(num_teams):
  routing.solver().Add(count.CumulVar(routing.End(i)) > 1)

Add interdependencies from exercise description


- one central headquarter (location 0)
- Every task is bound to a specific location
- minimize the overall travelling costs which are known in advance for every pair
- respect inter-dependencies among the tasks. For example: A needs to be completed before B
- nr of team as imput together with cost matrix (automatically created)

In [14]:
solver = routing.solver()

routing.AddConstantDimension(1, sys.maxsize, True, "time")
time = routing.GetDimensionOrDie("time")

# Write your code here ...
# Use time.CumulVar(...)
solver.Add(time.CumulVar(1) < time.CumulVar(2))
solver.Add(time.CumulVar(2) < time.CumulVar(4))
solver.Add(time.CumulVar(2) < time.CumulVar(11))

By default, the OR-Tools routing library does only approximate the optimal solution. For the verification of this exercise, however, you should find an optimal solution, which can be achieved by adapting the solver parameters as follows. Just leave this as it is. The 5 seconds are chosen so that testing is still efficient. Before verifying your result, change the time parameter to 30 seconds for a last run and check whether the result is still the same.

In [15]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.seconds = 5

Start solver

In [16]:
solution = routing.SolveWithParameters(search_parameters)

Print solution

In [17]:
def print_solution(vehicles, cities, manager, routing, solution):
    sum_route_distance = 0
    for vehicle_id in range(vehicles):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f"{cities[manager.IndexToNode(index)]} > "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += f"{cities[manager.IndexToNode(index)]}\n"
        plan_output += f"Total distance of vehicle {vehicle_id}: {route_distance} miles\n"
        print(plan_output)
        sum_route_distance += route_distance
    print(f"Total distance over all vehicles: {sum_route_distance} miles")

if solution:
    print_solution(num_teams, location_names, manager, routing, solution)
else:
    print("No solution found.")

Route for vehicle 0:
0 > 5 > 8 > 6 > 11 > 3 > 4 > 0
Total distance of vehicle 0: 986 miles

Route for vehicle 1:
0 > 7 > 1 > 2 > 10 > 9 > 0
Total distance of vehicle 1: 722 miles

Total distance over all vehicles: 1708 miles
